In [30]:
import time
import dateparser
import pytz
import json
import talib
import pandas as pd 
from datetime import datetime
from binance.client import Client
from keras.models import Sequential
from keras.layers import Dense

In [31]:
def date_to_milliseconds(date_str): # date to milliseconds
    """Convert UTC date to milliseconds
    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/
    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    :type date_str: str
    """
    # get epoch value in UTC
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d = dateparser.parse(date_str)
    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)

In [32]:
def interval_to_milliseconds(interval): # interval to milliseconds
    """Convert a Binance interval string to milliseconds
    :param interval: Binance interval string 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w
    :type interval: str
    :return:
         None if unit not one of m, h, d or w
         None if string not in correct format
         int value of interval in milliseconds
    """
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }

    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

In [33]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
    """Get Historical Klines from Binance
    See dateparse docs for valid start and end string formats http://dateparser.readthedocs.io/en/latest/
    If using offset strings for dates add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    :param symbol: Name of symbol pair e.g BNBBTC
    :type symbol: str
    :param interval: Biannce Kline interval
    :type interval: str
    :param start_str: Start date string in UTC format
    :type start_str: str
    :param end_str: optional - end date string in UTC format
    :type end_str: str
    :return: list of OHLCV values
    """
    # create the Binance client, no need for api key
    client = Client("", "")

    # init our list
    output_data = []

    # setup the max limit
    limit = 500

    # convert interval to useful value in seconds
    timeframe = interval_to_milliseconds(interval)

    # convert our date strings to milliseconds
    start_ts = date_to_milliseconds(start_str)

    # if an end time was passed convert it
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # it can be difficult to know when a symbol was listed on Binance so allow start time to be before list date
    symbol_existed = False
    while True:
        # fetch the klines from start_ts up to max 500 entries or the end_ts if set
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )

        # handle the case where our start date is before the symbol pair listed on Binance
        if not symbol_existed and len(temp_data):
            symbol_existed = True

        if symbol_existed:
            # append this loops data to our output data
            output_data += temp_data

            # update our start timestamp using the last value in the array and add the interval timeframe
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            # it wasn't listed yet, increment our start date
            start_ts += timeframe

        idx += 1
        # check if we received less than the required limit and exit the loop
        if len(temp_data) < limit:
            # exit the while loop
            break

        # sleep after every 3rd call to be kind to the API
        if idx % 3 == 0:
            time.sleep(1)

    return output_data


In [34]:
# setting desired dataset characteristics
symbol = "ETHUSDT" 
start = "1 Nov, 2019"
end = "1 Nov, 2021"
interval = Client.KLINE_INTERVAL_30MINUTE  # https://github.com/sammchardy/python-binance/blob/master/binance/client.py

# requesting dataset into pandas df 
columns = ['Open time', 'Open',
'High','Low','Close','Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore']
dataset = pd.DataFrame(get_historical_klines(symbol, interval, start, end), columns=columns)

/Users/neilperry/PycharmProjects/algo-trader/venv/lib/python3.8/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [36]:
# shape of data
dataset.shape

(35010, 12)

In [37]:
# look at head of data
dataset.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1572566400000,182.19000000,182.22000000,181.20000000,181.39000000,5265.95771000,1572568199999,956104.23521340,2909,2566.65305000,465982.18025310,0
1,1572568200000,181.39000000,181.96000000,181.29000000,181.91000000,4120.24802000,1572569999999,748907.09511060,2249,1742.04669000,316606.12959200,0
2,1572570000000,181.84000000,182.64000000,181.82000000,182.09000000,3980.20413000,1572571799999,725620.90650280,2404,1770.83312000,322812.40945340,0
3,1572571800000,182.10000000,182.50000000,181.55000000,181.55000000,2778.56939000,1572573599999,505752.66930280,2318,1384.92003000,252128.42816080,0
4,1572573600000,181.55000000,182.00000000,181.28000000,181.64000000,3789.02530000,1572575399999,688209.10129240,2592,1714.17791000,311379.42791150,0


In [11]:
# Observation Data Types
dataset.dtypes

Open time                        int64
Open                            object
High                            object
Low                             object
Close                           object
Volume                          object
Close time                       int64
Quote asset volume              object
Number of trades                 int64
Taker buy base asset volume     object
Taker buy quote asset volume    object
Ignore                          object
dtype: object

In [47]:
dataset= dataset.astype(float)
df1 = dataset["Close"].pct_change() * 100

In [50]:
df1 = df1.rename("Today")

In [51]:
df1 = df1.reset_index()

In [53]:
for i in range (1,6): 
    df1['Lag ' + str(i)] = df1["Today"].shift(i)

In [55]:
df1["index"] = dataset["Open time"]

In [56]:
df1

,index,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
0,1.572566e+12,NaN,NaN,NaN,NaN,NaN,NaN
1,1.572568e+12,0.286675,NaN,NaN,NaN,NaN,NaN
2,1.572570e+12,0.098950,0.286675,NaN,NaN,NaN,NaN
3,1.572572e+12,-0.296557,0.098950,0.286675,NaN,NaN,NaN
4,1.572574e+12,0.049573,-0.296557,0.098950,0.286675,NaN,NaN
...,...,...,...,...,...,...,...
35005,1.635718e+12,-0.004644,0.726483,0.241025,-0.026018,0.433163,-0.215410
35006,1.635719e+12,-0.307692,-0.004644,0.726483,0.241025,-0.026018,0.433163
35007,1.635721e+12,-0.098532,-0.307692,-0.004644,0.726483,0.241025,-0.026018
35008,1.635723e+12,-0.036374,-0.098532,-0.307692,-0.004644,0.726483,0.241025


In [57]:
df1['Volume'] = dataset["Volume"].shift(i).values/1000_000_000

In [58]:
df1

,index,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume
0,1.572566e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.572568e+12,0.286675,NaN,NaN,NaN,NaN,NaN,NaN
2,1.572570e+12,0.098950,0.286675,NaN,NaN,NaN,NaN,NaN
3,1.572572e+12,-0.296557,0.098950,0.286675,NaN,NaN,NaN,NaN
4,1.572574e+12,0.049573,-0.296557,0.098950,0.286675,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
35005,1.635718e+12,-0.004644,0.726483,0.241025,-0.026018,0.433163,-0.215410,0.000004
35006,1.635719e+12,-0.307692,-0.004644,0.726483,0.241025,-0.026018,0.433163,0.000006
35007,1.635721e+12,-0.098532,-0.307692,-0.004644,0.726483,0.241025,-0.026018,0.000004
35008,1.635723e+12,-0.036374,-0.098532,-0.307692,-0.004644,0.726483,0.241025,0.000005


In [63]:
df1 = df1.dropna()

In [61]:
df1["Direction"] = [1 if i > 0 else 0 for i in df1["Today"]]

In [66]:
df1

,index,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume,Direction,const
6,1.572577e+12,-0.895506,0.209205,0.049573,-0.296557,0.098950,0.286675,0.000004,0,1
7,1.572579e+12,-0.160763,-0.895506,0.209205,0.049573,-0.296557,0.098950,0.000004,0,1
8,1.572581e+12,0.294281,-0.160763,-0.895506,0.209205,0.049573,-0.296557,0.000003,1,1
9,1.572583e+12,0.033217,0.294281,-0.160763,-0.895506,0.209205,0.049573,0.000004,1,1
10,1.572584e+12,0.188168,0.033217,0.294281,-0.160763,-0.895506,0.209205,0.000005,1,1
...,...,...,...,...,...,...,...,...,...,...
35005,1.635718e+12,-0.004644,0.726483,0.241025,-0.026018,0.433163,-0.215410,0.000004,0,1
35006,1.635719e+12,-0.307692,-0.004644,0.726483,0.241025,-0.026018,0.433163,0.000006,0,1
35007,1.635721e+12,-0.098532,-0.307692,-0.004644,0.726483,0.241025,-0.026018,0.000004,0,1
35008,1.635723e+12,-0.036374,-0.098532,-0.307692,-0.004644,0.726483,0.241025,0.000005,0,1


In [84]:
len(df1)

35004

In [151]:
# X_train = df1[:11668][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume' ]].values
X_train = df1[:11668][['const','Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume' ]].values
y_train = df1[:11668][['Direction']].values

X_test1 = df1[11668:(11668 + 11668)][['const','Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume' ]].values
y_test1 = df1[11668:(11668 + 11668)][['Direction']].values

X_test2 = df1[11668 + 11668:][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume' ]].values
y_test2  = df1[11668 + 11668:][['Direction']].values

# Modeling  - [WORKING]

In [152]:
from keras.models import Sequential
from keras.layers import Dense

In [160]:
# input variables (X)
X = X_train
# output variables (y)
y = y_train

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(24, input_dim=7, activation='relu6')) # input_dim=12 : number of obervations in x
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(16, activation='relu6'))
model.add(Dense(1, activation='sigmoid'))
model

# Compile Keras Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# fit the keras model on the dataset
# model.fit(X, y, epochs=150, batch_size=2, verbose=0)
model.fit(X, y, epochs=150, batch_size=2)   # ! OVER-FIT

Epoch 1/150
5834/5834 [==============================] - 6s 976us/step - loss: 0.6904 - accuracy: 0.5313
Epoch 2/150
5834/5834 [==============================] - 5s 898us/step - loss: 0.6884 - accuracy: 0.5442
Epoch 3/150
5834/5834 [==============================] - 5s 894us/step - loss: 0.6878 - accuracy: 0.5457
Epoch 4/150
5834/5834 [==============================] - 5s 884us/step - loss: 0.6877 - accuracy: 0.5459
Epoch 5/150
5834/5834 [==============================] - 5s 865us/step - loss: 0.6873 - accuracy: 0.5506
Epoch 6/150
5834/5834 [==============================] - 5s 886us/step - loss: 0.6876 - accuracy: 0.5507
Epoch 7/150
5834/5834 [==============================] - 5s 892us/step - loss: 0.6871 - accuracy: 0.5486
Epoch 8/150
5834/5834 [==============================] - 5s 890us/step - loss: 0.6868 - accuracy: 0.5537
Epoch 9/150
5834/5834 [==============================] - 5s 889us/step - loss: 0.6865 - accuracy: 0.5522
Epoch 10/150
5834/5834 [==============================]

In [157]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test1, y_test1) # _, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

365/365 [==============================] - 0s 725us/step - loss: 0.6875 - accuracy: 0.5497
Accuracy: 54.97


In [158]:
# make class predictions with the model - round predictions
predictionsTest = (model.predict(X_test1) > 0.5).astype(int)

# summarize the first 5 cases
for i in range(10):
	print('%s => %d (expected %d)' % (X_test1[i].tolist(), predictions[i], y_test1[i]))

[1.0, 0.2251959637954215, 0.0997052193514758, -0.19469562583827038, 0.09527521545191764, -0.17724364516686908, 5.82936273e-06] => 0 (expected 0)
[1.0, -0.19012228319578162, 0.2251959637954215, 0.0997052193514758, -0.19469562583827038, 0.09527521545191764, 4.8123114400000005e-06] => 1 (expected 0)
[1.0, 0.0, -0.19012228319578162, 0.2251959637954215, 0.0997052193514758, -0.19469562583827038, 5.81197865e-06] => 1 (expected 0)
[1.0, -0.1645092861162767, 0.0, -0.19012228319578162, 0.2251959637954215, 0.0997052193514758, 6.569e-06] => 1 (expected 1)
[1.0, 0.10840813494643609, -0.1645092861162767, 0.0, -0.19012228319578162, 0.2251959637954215, 1.338038631e-05] => 1 (expected 1)
[1.0, 0.2772242917785661, 0.10840813494643609, -0.1645092861162767, 0.0, -0.19012228319578162, 6.505049020000001e-06] => 0 (expected 1)
[1.0, 0.043196544276447035, 0.2772242917785661, 0.10840813494643609, -0.1645092861162767, 0.0, 9.035715749999999e-06] => 0 (expected 0)
[1.0, -0.22884283246977777, 0.043196544276447035

In [159]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test2, y_test2) # _, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

365/365 [==============================] - 0s 764us/step - loss: 0.6945 - accuracy: 0.5193
Accuracy: 51.93


In [139]:
# make class predictions with the model - round predictions
predictionsTest = (model.predict(X_test2) > 0.5).astype(int)

# summarize the first 5 cases
for i in range(10):
	print('%s => %d (expected %d)' % (X_test2[i].tolist(), predictions[i], y_test2[i]))

[-0.20231494990760046, -0.0330597798607557, -0.42600983695441075] => 0 (expected 1)
[1.447015327836354, -0.20231494990760046, -0.0330597798607557] => 1 (expected 0)
[-0.19599052071991307, 1.447015327836354, -0.20231494990760046] => 1 (expected 1)
[0.450508265735694, -0.19599052071991307, 1.447015327836354] => 1 (expected 0)
[-0.0689981217177893, 0.450508265735694, -0.19599052071991307] => 1 (expected 0)
[-0.0038358756664913507, -0.0689981217177893, 0.450508265735694] => 0 (expected 0)
[-0.5236171137764201, -0.0038358756664913507, -0.0689981217177893] => 0 (expected 1)
[0.23779990102383852, -0.5236171137764201, -0.0038358756664913507] => 1 (expected 1)
[0.5103774613209433, 0.23779990102383852, -0.5236171137764201] => 1 (expected 0)
[-0.578595168379481, 0.5103774613209433, 0.23779990102383852] => 1 (expected 1)
